In [62]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import calendar

In [63]:
df = pd.read_excel('./ISPdatabase.xlsx')

In [64]:
vendor_mapping = pd.read_excel(r'C:\Medline\2. CPM\data\vendor_mapping\Vendor _mapping 2024_v1.xlsx')
vendor_mapping['Vendor Number'] = vendor_mapping['Vendor Number'].map(str)
vendor_mapping_dict = dict(zip(vendor_mapping['Vendor Number'],vendor_mapping['Cleaned Vendor Name']))


In [65]:
df_isp_buom_2022 = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\ISP_ShippingData_2022.xlsx',usecols='A,C,D,AA,BD')
df_isp_buom_2023 = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\ISP_ShippingData_2023.xlsx',usecols='A,C,D,AA,BD')
df_isp_buom = pd.concat([df_isp_buom_2023,df_isp_buom_2022],ignore_index=True)
df_isp_buom['VendorNumber'] = df_isp_buom['VendorNumber'].map(str)

In [66]:
vendor_mapping_inspection = vendor_mapping.loc[(~vendor_mapping['Regional Manager'].isin(['Exemption','US vendor']))&(vendor_mapping['Regional Manager'].notnull()),'Vendor Number'].to_list()
df_isp_buom = df_isp_buom.loc[df_isp_buom['VendorNumber'].isin(vendor_mapping_inspection)]
df_isp_buom['VendorName'] = df_isp_buom['VendorNumber'].apply(lambda x : vendor_mapping_dict.get(x,np.nan))

In [67]:
df_isp_buom

,SourceERPInd,ProductDivisionCode,VendorNumber,BUOMShipmentQty,ShipmentDate,VendorName
0,QADEU,21,147612,1440.0,2023-01-07,LIANYUNGANG ANSON NON WOVEN PRODUC
1,QADEU,21,147612,2880.0,2023-02-04,LIANYUNGANG ANSON NON WOVEN PRODUC
2,QADEU,21,147612,1080.0,2023-04-14,LIANYUNGANG ANSON NON WOVEN PRODUC
3,QADEU,21,147612,1080.0,2023-04-14,LIANYUNGANG ANSON NON WOVEN PRODUC
4,QADEU,21,147612,720.0,2023-07-03,LIANYUNGANG ANSON NON WOVEN PRODUC
...,...,...,...,...,...,...
150791,SAPCA,40,130324,174000.0,2022-10-04,HONGRAY (USA) MEDICAL PRODUCTS INC
150792,SAPCA,40,130324,690000.0,2022-12-10,HONGRAY (USA) MEDICAL PRODUCTS INC
150793,SAPCA,40,130324,690000.0,2022-12-10,HONGRAY (USA) MEDICAL PRODUCTS INC
150794,SAPCA,40,122928,110000.0,2022-07-27,BETTER HEALTH MEDICAL PRODUCTS HONG


In [68]:
df_isp_buom.rename(columns={'SourceERPInd': 'Market',
                                 'ProductDivisionCode':'Division',
                                 'BUOMShipmentQty':'BUOM'
                                 }, inplace=True)
df_isp_buom['Year'] = df_isp_buom['ShipmentDate'].dt.year
df_isp_buom['Month'] = df_isp_buom['ShipmentDate'].dt.month
df_isp_buom['Market'].replace({'QADEU':'EU',
                                    'MFGEU':'EU',
                                    'QADAU':'ANZ',
                                    'QADJP':'JAPAN'
                                    },inplace=True)
df_isp_buom = df_isp_buom.loc[df_isp_buom['Market'].isin(['EU','ANZ','JAPAN'])]

In [69]:
df_isp_buom_gy = df_isp_buom.groupby(['Market','VendorName','Division','Year','Month'],as_index=False)['BUOM'].sum()

In [70]:
df_gy = df.groupby(['Market','VendorName','Division','Year','Month'],as_index=False).size()

In [71]:
df_isp_buom_gy.merge(df_gy,how='outer',on=['Market','VendorName','Division','Year','Month']).fillna(0).to_excel('./out.xlsx')